In [9]:
!pip install nltk

import sys 
# Imported to handle natural language processing
import nltk
# Imported for machine learning classifiers
import sklearn
# Imported to store the data
import pandas
import numpy

print("To check the version:")
print("Python: {}".format(sys.version))
print("NLTK: {}".format(nltk.__version__))
print("Scikit-learn: {}".format(sklearn.__version__))
print("Pandas: {}".format(pandas.__version__))
print("Numpy: {}".format(numpy.__version__))

You should consider upgrading via the 'C:\Users\Niranjana S.Avilery\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


     ---------------------------------------- 1.5/1.5 MB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 96.6/96.6 KB 1.4 MB/s eta 0:00:00
     -------------------------------------- 262.0/262.0 KB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 78.4/78.4 KB 1.5 MB/s eta 0:00:00
To check the version:
Python: 3.10.4 (tags/v3.10.4:9d38120, Mar 23 2022, 23:13:41) [MSC v.1929 64 bit (AMD64)]
NLTK: 3.7
Scikit-learn: 1.1.0
Pandas: 1.4.2
Numpy: 1.22.3


## 1. Load the Dataset

In [ ]:
import pandas as pd
import numpy as np

